# Projeto 1 - Redes Sociais

## Minerando Dados

<!-- add gif with url 'https://i.gifer.com/origin/86/866369c0a0ff5b42080048d1986977ef_w200.gif'-->
[![gif](https://i.gifer.com/origin/86/866369c0a0ff5b42080048d1986977ef_w200.gif)](https://i.gifer.com/origin/86/866369c0a0ff5b42080048d1986977ef.gif)

## Alunos

<table>
  <tr>
    <td align="center"><a href="https://github.com/DaviReisVieira"><img style="border-radius: 50%;" src="https://github.com/DaviReisVieira.png" width="100px;" alt=""/><br /><sub><b>Davi Reis Vieira</b></sub></a><br /><a href="https://github.com/DaviReisVieira" title="Davi Reis Vieira">O Cara -1</a></td>
    <td align="center"><a href="https://github.com/guishas"><img style="border-radius: 50%;" src="https://github.com/guishas.png" width="100px;" alt=""/><br /><sub><b>Guilherme Lunetta</b></sub></a><br /><a href="https://github.com/guishas" title="Guilherme Lunetta">O cara 0</a></td>
    <td align="center"><a href="https://github.com/josermf2"><img style="border-radius: 50%;" src="https://github.com/josermf2.png" width="100px;" alt=""/><br /><sub><b>José Fernandes</b></sub></a><br /><a href="https://github.com/josermf2" title="José Fernandes">O cara +1</a></td>
  </tr>
</table>

## Importando as bibliotecas necessárias

Vamos importar as bibliotecas necessárias:

In [1]:
# Para utilizar recursos do sistema
import os
import sys

# Bibliotecas DASK
import dask
from dask.distributed import Client
import dask.dataframe as dd # similar ao pandas
import dask.multiprocessing

# Para plot
%matplotlib inline
import matplotlib.pyplot as plt

Caso obtenha algum erro, utilize o **!pip install** para instalar a biblioteca ausente!

Vamos conferir com qual versão da biblioteca **Dask** estamos trabalhando?

In [2]:
dask.__version__

'2022.10.0'

Você também pode conferir de onde está executando o Python e qual a versão

In [3]:
print('Executável:')
print(sys.executable)

print('\nVersão do Python:')
print(sys.version)

Executável:
c:\Users\davi.souza\Desktop\Personal\redes-sociais\projeto-1-redesoc\venv\Scripts\python.exe

Versão do Python:
3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)]


Vamos conferir em qual diretório iremos trabalhar (é o diretório do notebook)

In [4]:
print('O seu notebook está na pasta:')
print(os.getcwd())

O seu notebook está na pasta:
c:\Users\davi.souza\Desktop\Personal\redes-sociais\projeto-1-redesoc


## Configurando um ambiente Dask

Agora iremos configurar um ambiente Dask para execução das nossas análises. Para isso, iremos criar um **client**

!pip install 'dask[complete]==2022.8.0'

!pip install dask

!pip install 'dask[dataframe]'

In [6]:
client = Client(n_workers=5, threads_per_worker=3, memory_limit='4GB')

c:\Users\davi.souza\Desktop\Personal\redes-sociais\projeto-1-redesoc\venv\lib\site-packages\distributed\node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 52159 instead
  warnings.warn(


perceba que, ao exibir o client, temos acesso a uma URL que nos permitirá acompanhar informações sobre o consumo de recursos durante a execução de tarefas

In [6]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 7.45 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:53026,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 7.45 GiB
Comm: tcp://127.0.0.1:53059,Total threads: 2
Dashboard: http://127.0.0.1:53060/status,Memory: 1.86 GiB
Nanny: tcp://127.0.0.1:53032,


In [7]:
reviews = dd.read_csv('steam_reviews.csv',
                      sep=',',
                      encoding='UTF-8',
                      on_bad_lines='skip',
                      engine='python',
                     dtype={'Unnamed: 0': 'object',
                           'app_id': 'object',
                           'author.num_games_owned': 'object',
                           'author.num_reviews': 'object',
                           'author.steamid': 'object',
                           'review_id': 'object',
                            'recommended': 'bool',
                           'timestamp_created': 'object',
                           'timestamp_updated': 'object',
                           'comment_count': 'object',
                           'weighted_vote_score': 'object',
                           'votes_funny': 'object',
                           'votes_helpful': 'object'})

## Amostra dos dados

Semelhante ao Pandas, podemos solicitar uma pequena amostra dos dados com

In [8]:
reviews = reviews.drop(["Unnamed: 0"], axis=1)
reviews.head(5)

,app_id,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,...,steam_purchase,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
0,292030,The Witcher 3: Wild Hunt,85185598,schinese,不玩此生遗憾，RPG游戏里的天花板，太吸引人了,1611381629,1611381629,True,0,0,...,True,False,False,76561199095369542,6,2,1909.0,1448.0,1909.0,1.611343e+09
1,292030,The Witcher 3: Wild Hunt,85185250,schinese,拔DIAO无情打桩机--杰洛特!!!,1611381030,1611381030,True,0,0,...,True,False,False,76561198949504115,30,10,2764.0,2743.0,2674.0,1.611386e+09
2,292030,The Witcher 3: Wild Hunt,85185111,schinese,巫师3NB,1611380800,1611380800,True,0,0,...,True,False,False,76561199090098988,5,1,1061.0,1061.0,1060.0,1.611384e+09
3,292030,The Witcher 3: Wild Hunt,85184605,english,"One of the best RPG's of all time, worthy of a...",1611379970,1611379970,True,0,0,...,True,False,False,76561199054755373,5,3,5587.0,3200.0,5524.0,1.611384e+09
4,292030,The Witcher 3: Wild Hunt,85184287,schinese,大作,1611379427,1611379427,True,0,0,...,True,False,False,76561199028326951,7,4,217.0,42.0,217.0,1.610788e+09


In [9]:
reviews.dtypes

app_id                             object
app_name                           object
review_id                          object
language                           object
review                             object
timestamp_created                  object
timestamp_updated                  object
recommended                          bool
votes_helpful                      object
votes_funny                        object
weighted_vote_score                object
comment_count                      object
steam_purchase                       bool
received_for_free                    bool
written_during_early_access          bool
author.steamid                     object
author.num_games_owned             object
author.num_reviews                 object
author.playtime_forever           float64
author.playtime_last_two_weeks    float64
author.playtime_at_review         float64
author.last_played                float64
dtype: object

In [10]:
reviews_ptbr = reviews[reviews.language == 'brazilian']
reviews_ptbr.head(5)

,app_id,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,...,steam_purchase,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
29,292030,The Witcher 3: Wild Hunt,85177505,brazilian,Se um dia alguém falar que esse jogo é ruim na...,1611368498,1611368498,True,1,0,...,True,False,False,76561198844659805,70,4,11115.0,2252.0,11115.0,1.611186e+09
32,292030,The Witcher 3: Wild Hunt,85176661,brazilian,NaN,1611367193,1611367193,True,0,0,...,True,False,False,76561198076880796,127,13,875.0,752.0,826.0,1.611370e+09
34,292030,The Witcher 3: Wild Hunt,85176249,brazilian,Obra prima!!!,1611366524,1611366524,True,0,0,...,True,False,False,76561198957873353,32,1,2888.0,1475.0,2888.0,1.611366e+09
43,292030,The Witcher 3: Wild Hunt,85173023,brazilian,Jogão da porra.,1611361229,1611361229,True,0,0,...,True,False,False,76561198141110905,59,4,20193.0,3692.0,20193.0,1.611297e+09
48,292030,The Witcher 3: Wild Hunt,85172417,brazilian,very foda,1611360271,1611360271,True,0,0,...,True,False,False,76561198107351100,60,5,2104.0,2037.0,1882.0,1.611386e+09


In [11]:
reviews_ptbr['timestamp_created'] = dd.to_datetime(reviews_ptbr['timestamp_created'], unit='s')
reviews_ptbr['timestamp_updated'] = dd.to_datetime(reviews_ptbr['timestamp_updated'], unit='s')
reviews_ptbr.head(5)

,app_id,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,...,steam_purchase,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
29,292030,The Witcher 3: Wild Hunt,85177505,brazilian,Se um dia alguém falar que esse jogo é ruim na...,2021-01-23 02:21:38,2021-01-23 02:21:38,True,1,0,...,True,False,False,76561198844659805,70,4,11115.0,2252.0,11115.0,1.611186e+09
32,292030,The Witcher 3: Wild Hunt,85176661,brazilian,NaN,2021-01-23 01:59:53,2021-01-23 01:59:53,True,0,0,...,True,False,False,76561198076880796,127,13,875.0,752.0,826.0,1.611370e+09
34,292030,The Witcher 3: Wild Hunt,85176249,brazilian,Obra prima!!!,2021-01-23 01:48:44,2021-01-23 01:48:44,True,0,0,...,True,False,False,76561198957873353,32,1,2888.0,1475.0,2888.0,1.611366e+09
43,292030,The Witcher 3: Wild Hunt,85173023,brazilian,Jogão da porra.,2021-01-23 00:20:29,2021-01-23 00:20:29,True,0,0,...,True,False,False,76561198141110905,59,4,20193.0,3692.0,20193.0,1.611297e+09
48,292030,The Witcher 3: Wild Hunt,85172417,brazilian,very foda,2021-01-23 00:04:31,2021-01-23 00:04:31,True,0,0,...,True,False,False,76561198107351100,60,5,2104.0,2037.0,1882.0,1.611386e+09


In [15]:
reviews_july = reviews_ptbr[reviews_ptbr['timestamp_created'].dt.month == 7]
reviews_july.head(5)

,app_id,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,...,steam_purchase,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
73075,292030,The Witcher 3: Wild Hunt,73579864,brazilian,"I'll only speak about performance, since we al...",2020-07-31 22:19:22,2020-07-31 22:19:22,True,0,0,...,True,False,False,76561198074616040,148,12,29259.0,0.0,18588.0,1.597875e+09
73093,292030,The Witcher 3: Wild Hunt,73576952,brazilian,Não tenho palavras para dizer como esse jogo e...,2020-07-31 20:46:46,2020-07-31 20:48:04,True,1,0,...,True,False,False,76561198120075031,38,9,4389.0,0.0,4389.0,1.557089e+09
73101,292030,The Witcher 3: Wild Hunt,73575959,brazilian,Epic,2020-07-31 20:17:38,2020-07-31 20:17:38,True,0,0,...,True,False,False,76561198101494872,25,1,6653.0,0.0,4136.0,1.608502e+09
73103,292030,The Witcher 3: Wild Hunt,73575666,brazilian,"Melhor jogo do fucking universo, compre isso e...",2020-07-31 20:09:24,2020-07-31 20:09:24,True,1,0,...,True,False,False,76561198032616918,80,2,24894.0,0.0,18568.0,1.608581e+09
73113,292030,The Witcher 3: Wild Hunt,73574848,brazilian,Um dos melhores games já feito.,2020-07-31 19:45:16,2020-07-31 19:45:16,True,1,0,...,True,False,False,76561198276339634,37,4,7796.0,0.0,7755.0,1.596227e+09


In [16]:
reviews_july.dtypes

app_id                                    object
app_name                                  object
review_id                                 object
language                                  object
review                                    object
timestamp_created                 datetime64[ns]
timestamp_updated                 datetime64[ns]
recommended                                 bool
votes_helpful                             object
votes_funny                               object
weighted_vote_score                       object
comment_count                             object
steam_purchase                              bool
received_for_free                           bool
written_during_early_access                 bool
author.steamid                            object
author.num_games_owned                    object
author.num_reviews                        object
author.playtime_forever                  float64
author.playtime_last_two_weeks           float64
author.playtime_at_r

In [17]:
# convert reviews_july to pd.DataFrame
reviews_july_pd = reviews_july.compute()
reviews_july_pd.head(5)

,app_id,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,...,steam_purchase,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
73075,292030,The Witcher 3: Wild Hunt,73579864,brazilian,"I'll only speak about performance, since we al...",2020-07-31 22:19:22,2020-07-31 22:19:22,True,0,0,...,True,False,False,76561198074616040,148,12,29259.0,0.0,18588.0,1.597875e+09
73093,292030,The Witcher 3: Wild Hunt,73576952,brazilian,Não tenho palavras para dizer como esse jogo e...,2020-07-31 20:46:46,2020-07-31 20:48:04,True,1,0,...,True,False,False,76561198120075031,38,9,4389.0,0.0,4389.0,1.557089e+09
73101,292030,The Witcher 3: Wild Hunt,73575959,brazilian,Epic,2020-07-31 20:17:38,2020-07-31 20:17:38,True,0,0,...,True,False,False,76561198101494872,25,1,6653.0,0.0,4136.0,1.608502e+09
73103,292030,The Witcher 3: Wild Hunt,73575666,brazilian,"Melhor jogo do fucking universo, compre isso e...",2020-07-31 20:09:24,2020-07-31 20:09:24,True,1,0,...,True,False,False,76561198032616918,80,2,24894.0,0.0,18568.0,1.608581e+09
73113,292030,The Witcher 3: Wild Hunt,73574848,brazilian,Um dos melhores games já feito.,2020-07-31 19:45:16,2020-07-31 19:45:16,True,1,0,...,True,False,False,76561198276339634,37,4,7796.0,0.0,7755.0,1.596227e+09


In [18]:
# reviews_july_pd to csv
reviews_july_pd.to_csv('reviews_july.csv', index=False)

In [19]:
# convert reviews_ptbr to pd.DataFrame
reviews_ptbr_pd = reviews_ptbr.compute()
reviews_ptbr_pd.head(5)

,app_id,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,...,steam_purchase,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
29,292030,The Witcher 3: Wild Hunt,85177505,brazilian,Se um dia alguém falar que esse jogo é ruim na...,2021-01-23 02:21:38,2021-01-23 02:21:38,True,1,0,...,True,False,False,76561198844659805,70,4,11115.0,2252.0,11115.0,1.611186e+09
32,292030,The Witcher 3: Wild Hunt,85176661,brazilian,NaN,2021-01-23 01:59:53,2021-01-23 01:59:53,True,0,0,...,True,False,False,76561198076880796,127,13,875.0,752.0,826.0,1.611370e+09
34,292030,The Witcher 3: Wild Hunt,85176249,brazilian,Obra prima!!!,2021-01-23 01:48:44,2021-01-23 01:48:44,True,0,0,...,True,False,False,76561198957873353,32,1,2888.0,1475.0,2888.0,1.611366e+09
43,292030,The Witcher 3: Wild Hunt,85173023,brazilian,Jogão da porra.,2021-01-23 00:20:29,2021-01-23 00:20:29,True,0,0,...,True,False,False,76561198141110905,59,4,20193.0,3692.0,20193.0,1.611297e+09
48,292030,The Witcher 3: Wild Hunt,85172417,brazilian,very foda,2021-01-23 00:04:31,2021-01-23 00:04:31,True,0,0,...,True,False,False,76561198107351100,60,5,2104.0,2037.0,1882.0,1.611386e+09
